In [3]:
import numpy as np
import rocketcea

In [5]:
# Define Inputs

#Engine Core Properties
A = 0.005771714879 #Chamber Area [m^2]
P_c= 3895540 #Chamber Pressure [Pa]
A_t = 0.0006575790181 #Throat Area [m^2]

g = 9.81 #Gravitational Acceleration [m/s^2]
R = 8.314 #Universal Gas Constant [J/mol*K]
pi = 3.14159265359

#Chamber Gas/Geometric Properties
G_c = 1.2409 #Chamber Gas Specific Heat Ratio
T_c = 3048.54 #Chamber Gas Temperature [K]
M_c = 0.019615 #Chamber Gas Molecular Weight [kg/mol]
Cp_c = 2183.2 #Chamber Gas Specific Heat Capacity [J/kg*K]
mu_c = 0.000092148 #Chamber Gas Viscosity [Pa*s]
k_c = 0.35289 #Chamber Gas Thermal Conductivity [W/m*K]
Pr_c = (Cp_c * mu_c)/(k_c) #Chamber Gas Prandtl Number
R_bar_c = R/M_c #Chamber Gas Specific Gas Constant [J/kg*K]

cstar = ((R_bar_c*T_c/G_c) * ((G_c+1)/2)**((G_c+1)/(G_c-1)))**(0.5) #Characteristic Velocity [m/s]
D_t = 2 * (A_t/pi)**(0.5) #Throat Diameter [m]
r_u = 0.00635 #Throat Radius of Curvature [m]

In [18]:
# Bartz Iterator Function Definition
# Initial Condition

def Bartz_Iterator(T_c, T_p, Ma, A, h_p, k_w, t_w):
    T_hwi = 500 #Hot Wall Temperature Initial Guess [K]
    T_res = 1

    while T_res > 0.001:
        T_old = T_hwi
            
        r = Pr_c**0.33 #Recovery Factor
        T_aw = T_c * (((1+(r*(G_c-1)/2)*Ma**2))/((1+((G_c-1)/2)*Ma**2))) #Local Adiabatic Wall Temperature [K]
        s = 1/((((0.5*T_hwi/T_c)*(1+((G_c-1)/2)*Ma**2)+0.5)**0.68)*(1+((G_c-1)/2)*Ma**2)**0.12) #Boundary Layer Correction Factor
        h_g = ((0.026/(D_t**0.2))*((mu_c**0.2) * Cp_c/(Pr_c**0.6))*(P_c/cstar)**0.8 * ((D_t/r_u)**0.1))*((A_t/A)**0.9)*s #Bartz Heat Transfer Coefficient [W/m^2*K]
        q = h_g*(T_aw - T_hwi)

        T_cw = (q/h_p) + T_p
        T_hwi = q*t_w/k_w + T_cw
        T_res = T_hwi-T_old
    return(T_hwi)

Bartz_Iterator(T_c = T_c, T_p = 330, Ma = 0, A = A, h_p = 17435.70437, k_w = 160, t_w = 0.00381)

789.4597406555492

In [20]:
#Define Propellant Inputs

m_dot = 2 #Mass Flow Rate [kg/s]
D = 0.004 #Channel Hydraulic Diameter [m]
Cp_p = 2010 #Propellant Specific Heat Capacity [J/kg*K] #Function of Temperature
k_p = 0.145 #Propellant Thermal Conductivity [W/m*k] #Function of Temperature
T_hwi = 789.4597406 #Hot Wall Initial Temperature [K]

In [ ]:
#Propellant Iterator Function Definition

def Propellant_Iterator(T_hwi, T_p, Ma, A, k_w, t_w, m_dot, D, x):
    T_px = 430 #Propellant Temperature Initial Guess [K]
    T_pres = 1

    while T_pres > 0.001:
        T_pold = T_px

        mu_px = 0.001832*np.exp(-3.0716 + 1.373*(293.15/T_px) + 1.6961*((293.15/T_px)**2)) #Subject to Accuracy Check
        Re_Dx = 4*(m_dot)/(3.14*D*mu_px) #Propellant Reynolds Number
        Pr_px = mu_px*Cp_p #Propellant Prandtl Number
        h_px = (k_p/D)*(0.023*(Re_Dx**0.8) * (Pr_px**0.4))

        T_hwx = 500 #Initial Hot Wall Temperature [K]
        T_res = 1

        #Bartz Iterator
        while T_res > 0.001:
            T_old = T_hwx
            
            r = Pr_c**0.33 #Recovery Factor
            T_aw = T_c * (((1+(r*(G_c-1)/2)*Ma**2))/((1+((G_c-1)/2)*Ma**2))) #Local Adiabatic Wall Temperature [K]
            s = 1/((((0.5*T_hwx/T_c)*(1+((G_c-1)/2)*Ma**2)+0.5)**0.68)*(1+((G_c-1)/2)*Ma**2)**0.12) #Boundary Layer Correction Factor
            h_gx = ((0.026/(D_t**0.2))*((mu_c**0.2) * Cp_c/(Pr_c**0.6))*(P_c/cstar)**0.8 * ((D_t/r_u)**0.1))*((A_t/A)**0.9)*s #Bartz Heat Transfer Coefficient [W/m^2*K]
            q = h_gx*(T_aw - T_hwx)

            T_cwx = (q/h_px) + T_p
            T_hwx = q*t_w/k_w + T_cwx
            T_res = T_hwx-T_old

        T_barhw = (T_hwi + T_hwx)/2
        T_barp = (T_px + T_p)/2

        #Average Condition
        s_bar = 1/((((0.5*T_barhw/T_c)*(1+((G_c-1)/2)*Ma**2)+0.5)**0.68)*(1+((G_c-1)/2)*Ma**2)**0.12) #Average Boundary Layer Correction Factor
        h_barg = ((0.026/(D_t**0.2))*((mu_c**0.2) * Cp_c/(Pr_c**0.6))*(P_c/cstar)**0.8 * ((D_t/r_u)**0.1))*((A_t/A)**0.9)*s #Average Bartz Heat Transfer Coefficient [W/m^2*K]

        mu_barp = 0.001832*np.exp(-3.0716 + 1.373*(293.15/T_barp) + 1.6961*((293.15/T_barp)**2)) #Subject to Accuracy Check
        Re_barD = 4*(m_dot)/(3.14*D*mu_barp) #Average Propellant Reynolds Number
        Pr_barp = mu_barp*Cp_p #Average Propellant Prandtl Number
        h_barp = (k_p/D)*(0.023*(Re_barD**0.8) * (Pr_barp**0.4))

        U_bar = 1/(1/(h_barg) + (t_w/k_w) + 1/h_barp)
        T_px = np.exp(-(6.28*D*U_bar)/(m_dot*Cp_p))*(T_p - T_c) + T_c
        T_pres = T_px - T_pold
    return(T_hwx)

Propellant_Iterator(T_hwi = T_hwi, T_p = 330,  Ma = 0, A = A, k_w = 160, t_w = 0.00381, m_dot = m_dot, D = D, x = 0) #m_dot=place holder value






np.float64(598.1067211547152)